In [12]:
%load_ext autoreload
%autoreload 2

import teetool as tt
import numpy as np

In [8]:
# generate trajectory data
cluster_data_1 = tt.helpers.get_trajectories(ntype=0,
                                             ndim=2,
                                             ntraj=500,
                                             npoints=100,
                                             noise_std=0.0)

cluster_data_2 = tt.helpers.get_trajectories(ntype=1,
                                             ndim=2,
                                             ntraj=500,
                                             npoints=100,
                                             noise_std=0.0)

In [9]:
# create a world
world = tt.World(name="toy", ndim=2, resolution=[100, 100])

# add data
world.addCluster(cluster_data_1, "one")
# world.addCluster(cluster_data_2, "two")

In [10]:
visual = tt.visual_2d.Visual_2d(world, dpi=300)

visual.plotTrajectories(ntraj=50)
visual.show()

In [20]:
# build the model
settings = {"model_type":"ML",
            "basis_type":"bernstein",
            "nbasis": 5,
            "ngaus":100}

world.buildModel(settings)

In [21]:
# x = np.array([0.5, 0.2])
# y = np.array([[0.5], [0.2], [0.2], [0.3]])

x = np.array([0.5])
y = np.array([[0.5], [0.5]])

world.addObservation(icluster=0, observation = (x, y) )

In [22]:
# visual
visual = tt.visual_2d.Visual_2d(world, dpi=300)

visual.plotTube(sdwidth=2)
visual.plotSamples(ntraj=10)

visual.show()